In [5]:
import pandas as pd

In [6]:
#Load the Datasets
bank=pd.read_csv(r"D:\AJAI\DATA ANALYTICS\banking\bank_data.csv")
print("Bank:", bank.shape)
cust=pd.read_csv(r"D:\AJAI\DATA ANALYTICS\banking\customer_data.csv")
print("Customers:", cust.shape)
trans=pd.read_csv(r"D:\AJAI\DATA ANALYTICS\banking\transaction_data.csv", parse_dates=['Transaction_Date'])
print("Transactions:", trans.shape)

Bank: (1000, 6)
Customers: (10000, 7)
Transactions: (10000, 8)


---BASIC CHECKS - FINDING NULL COUNTS, DUPLICATES AND TRANSACTION TIMEFRAME---

In [7]:
print(bank.head())
print(cust.head())
print(trans.head())

   Branch_ID     City Region  Firm_Revenue  Expenses  Profit_Margin
0       1000  Solapur   East      422443.0    346471          56.31
1       1001   Mumbai  North      211155.0    216256          79.53
2       1002   Nashik   West      985006.0     71777          55.31
3       1003  Solapur  North       55113.0    253655           2.57
4       1004   Nagpur   West      282701.0    148194         -14.10
   Customer_ID   Age Customer_Type      City Region  Bank_Name  Branch_ID
0       200000  56.0      Employee      Pune  North  HDFC Bank       1350
1       200001  69.0           NaN  Kolhapur   East  HDFC Bank       1648
2       200002  46.0      Business      Pune   West  HDFC Bank       1798
3       200003  32.0    Individual      Pune   West  HDFC Bank       1900
4       200004   NaN      Business  Kolhapur   East  HDFC Bank       1227
   Transaction_ID  Customer_ID Account_Type  Total_Balance  \
0          300000       209689     Business          69339   
1          300001       

In [8]:
print(bank.describe(include='all'))
print(cust.describe(include='all'))
print(trans.describe(include='all'))

          Branch_ID      City Region   Firm_Revenue      Expenses  \
count   1000.000000      1000   1000     950.000000    1000.00000   
unique          NaN         6      4            NaN           NaN   
top             NaN  Kolhapur   East            NaN           NaN   
freq            NaN       176    265            NaN           NaN   
mean    1499.500000       NaN    NaN  514074.181053  265325.99100   
std      288.819436       NaN    NaN  278665.374536  140572.96154   
min     1000.000000       NaN    NaN   51241.000000   20520.00000   
25%     1249.750000       NaN    NaN  260637.500000  145412.75000   
50%     1499.500000       NaN    NaN  518348.500000  265404.00000   
75%     1749.250000       NaN    NaN  746103.750000  394530.50000   
max     1999.000000       NaN    NaN  999975.000000  499568.00000   

        Profit_Margin  
count     1000.000000  
unique            NaN  
top               NaN  
freq              NaN  
mean        24.561410  
std         42.757352  
min

In [9]:
print(bank.info())
print(bank.isna().sum().sort_values(ascending=False).head(15),'\n**************\n')
print(cust.info())
print(cust.isna().sum().sort_values(ascending=False).head(15),'\n**************\n')
print(trans.info())
print(trans.isna().sum().sort_values(ascending=False).head(15))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Branch_ID      1000 non-null   int64  
 1   City           1000 non-null   object 
 2   Region         1000 non-null   object 
 3   Firm_Revenue   950 non-null    float64
 4   Expenses       1000 non-null   int64  
 5   Profit_Margin  1000 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 47.0+ KB
None
Firm_Revenue     50
Branch_ID         0
City              0
Region            0
Expenses          0
Profit_Margin     0
dtype: int64 
**************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Customer_ID    10000 non-null  int64  
 1   Age            9500 non-null   float64
 2   Customer_Type  9500 non-null   object

In [10]:
trans.duplicated().sum()
cust.duplicated().sum()
bank.duplicated().sum()

np.int64(0)

In [11]:
#Checking duplicates in transaction and customer ids
print("Duplicate Transaction IDs:", trans.duplicated(subset=['Transaction_ID']).sum())
print("Unique customers in customers file:", cust['Customer_ID'].nunique())
print("Unique customers in transactions file:", trans['Customer_ID'].nunique())

Duplicate Transaction IDs: 0
Unique customers in customers file: 10000
Unique customers in transactions file: 6335


In [12]:
#Finding the transaction range
print("Transaction date range:", trans['Transaction_Date'].min(), "to", trans['Transaction_Date'].max())

Transaction date range: 2022-03-21 00:00:00 to 2025-03-20 00:00:00


From findings there are no duplicate rows


In [13]:
# Changing Transaction Amount column to Numeric
trans['Transaction_Amount'] = pd.to_numeric(trans['Transaction_Amount'], errors='coerce')

# Convert to float64 and round to 2 decimal places
trans['Transaction_Amount'] = trans['Transaction_Amount'].astype('float64').round(2)

DATA CLEANING - (There are no duplicate rows in all three column)
1 Standardizing column names
2 Handle missing values

In [14]:
#Standardizing column names
bank.columns  = bank.columns.str.strip()
cust.columns  = cust.columns.str.strip()
trans.columns = trans.columns.str.strip()

2. Handling missing values 
From bank :
Firm_Revenue     50

From cust :
Age              500
Customer_Type    500
City             500

From trans:
No duplicates


In [15]:
bank['Firm_Revenue'] = bank['Firm_Revenue'].fillna(bank['Firm_Revenue'].median())
print(bank['Firm_Revenue'].isna().sum())
#print(bank['Firm_Revenue'].info())

0


In [16]:
cust['Age']=cust['Age'].fillna(cust['Age'].median())
print(cust['Age'].isna().sum())

0


In [17]:
cust['Customer_Type']=cust['Customer_Type'].fillna('Unknown')
print(cust['Customer_Type'].isna().sum())

0


In [18]:
cust['City'] = cust['City'].fillna(cust['City'].mode()[0])
print(cust['City'].isna().sum())

0


In [19]:
#Merging all 3 tables into one
df = trans.merge(cust, on='Customer_ID', how='left', suffixes=('','_cust'))
df = df.merge(bank, on='Branch_ID', how='left', suffixes=('','_branch'))
print(df.head())

   Transaction_ID  Customer_ID Account_Type  Total_Balance  \
0          300000       209689     Business          69339   
1          300001       206124     Business          12825   
2          300002       207501      Current          67753   
3          300003       208675      Savings          67061   
4          300004       204923     Business           8566   

   Transaction_Amount  Investment_Amount Investment_Type Transaction_Date  \
0              4794.0              42580   Fixed Deposit       2024-12-08   
1              3500.0              46605   Fixed Deposit       2022-09-09   
2              2401.0              17027   Fixed Deposit       2022-04-28   
3              2952.0               3054   Fixed Deposit       2023-05-22   
4              1025.0              44937   Fixed Deposit       2023-04-05   

    Age Customer_Type      City Region  Bank_Name  Branch_ID City_branch  \
0  68.0    Individual    Nashik  North  HDFC Bank       1928      Nashik   
1  45.0     

In [20]:
#Changing column to lowercase
df.columns = df.columns.str.lower()
print(df.head())

   transaction_id  customer_id account_type  total_balance  \
0          300000       209689     Business          69339   
1          300001       206124     Business          12825   
2          300002       207501      Current          67753   
3          300003       208675      Savings          67061   
4          300004       204923     Business           8566   

   transaction_amount  investment_amount investment_type transaction_date  \
0              4794.0              42580   Fixed Deposit       2024-12-08   
1              3500.0              46605   Fixed Deposit       2022-09-09   
2              2401.0              17027   Fixed Deposit       2022-04-28   
3              2952.0               3054   Fixed Deposit       2023-05-22   
4              1025.0              44937   Fixed Deposit       2023-04-05   

    age customer_type      city region  bank_name  branch_id city_branch  \
0  68.0    Individual    Nashik  North  HDFC Bank       1928      Nashik   
1  45.0     

In [21]:
# Defining the desired order of columns
new_column_order = [
    'transaction_id', 
    'customer_id', 
    'branch_id', 
    'transaction_amount', 
    'investment_amount', 
    'total_balance',
    'firm_revenue',
    'expenses',
    'profit_margin',
    'account_type',
    'investment_type',
    'customer_type',
    'age',
    'transaction_date',
    'city', 
    'region',
    'bank_name', 
    'city_branch', 
    'region_branch'
]

# Rearranging the columns with the desired order of columns
df = df[new_column_order]

# Display the new column order to confirm
print(df.columns)

print(df.head())

Index(['transaction_id', 'customer_id', 'branch_id', 'transaction_amount',
       'investment_amount', 'total_balance', 'firm_revenue', 'expenses',
       'profit_margin', 'account_type', 'investment_type', 'customer_type',
       'age', 'transaction_date', 'city', 'region', 'bank_name', 'city_branch',
       'region_branch'],
      dtype='object')
   transaction_id  customer_id  branch_id  transaction_amount  \
0          300000       209689       1928              4794.0   
1          300001       206124       1132              3500.0   
2          300002       207501       1690              2401.0   
3          300003       208675       1032              2952.0   
4          300004       204923       1653              1025.0   

   investment_amount  total_balance  firm_revenue  expenses  profit_margin  \
0              42580          69339      339233.0    477740          65.87   
1              46605          12825       83336.0    335733          47.84   
2              17027    

In [22]:
#Saving the new Dataset as cleaned.csv
df.to_csv(r'D:\AJAI\DATA ANALYTICS\banking\cleaned.csv',index=False)